<a href="https://colab.research.google.com/github/AllanKamimura/Pai_ta_ON_python/blob/main/2021_SME0340_ODE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#@title ## Condições Analisadas { run: "auto", form-width: "20%", display-mode: "form" }
#@markdown $A \rightleftharpoons {B} \rightarrow C$

#@markdown ---
#@markdown ### Selecione a Aproximação
#@markdown **Estado Quase Estacionario:** k2 >> km1 > k1 \
#@markdown **Aproximacao Pre-Equilibrio:** k1 > km1 >> k2 \
#@markdown **k_-1 proximo de 0:** k1 = k2 >> km1 \

options = "k_-1 proximo de 0" #@param ["Estado Quase Estacionario", "Aproximacao Pre-Equilibrio", "k_-1 proximo de 0"]
#@markdown ---

if options == "Estado Quase Estacionario":
    A0 = 1
    B0 = 0
    C0 = 0

    k1 = 1
    km1 = 10
    k2 = 1000

if options == "Aproximacao Pre-Equilibrio":
    A0 = 1
    B0 = 0
    C0 = 0

    k1 = 200
    km1 = 100
    k2 = 1

if options == "k_-1 proximo de 0":
    A0 = 1
    B0 = 0
    C0 = 0

    k1 = 1
    km1 = 0.0001
    k2 = 1

import sympy

t = sympy.symbols("t")

lamb1 = sympy.symbols("lambda1")
lamb2 = sympy.symbols("lambda2")
lamb3 = sympy.symbols("lambda3")

A = sympy.Function("A")
B = sympy.Function("B")
C = sympy.Function("C")

X = sympy.Matrix([A(t),B(t),C(t)])
dX = sympy.diff(X, t)

M = sympy.Matrix([[-k1, km1, 0], [k1, -k2-km1, 0], [0, k2, 0]])

xh = sympy.solvers.ode.dsolve(dX - M*X, ics = {A(0) : A0, B(0) : B0, C(0) : C0})

import numpy as np

def matrix(t_eval):
    return np.array([sympy.lambdify(t, xh[0].rhs, "numpy")(t_eval),
                     sympy.lambdify(t, xh[1].rhs, "numpy")(t_eval),
                     sympy.lambdify(t, xh[2].rhs, "numpy")(t_eval)])

import pandas as pd

df = pd.DataFrame(
    map(matrix, np.linspace(0,5,50)),
    columns = ["A(t)", "B(t)", "C(t)"]
    )

import plotly.graph_objects as go

fig = go.Figure(layout_title_text="Gráfico {}: Concetração pelo tempo".format(options))

fig.add_trace(go.Scatter(y = df["A(t)"],
                    mode = "lines+markers", 
                    name = "A(t)"))

fig.add_trace(go.Scatter(y = df["B(t)"],
                    mode = "lines+markers", 
                    name = "B(t)"))

fig.add_trace(go.Scatter(y = df["C(t)"],
                    mode = "lines+markers", 
                    name = "C(t)"))

fig.update_xaxes(title_text = "tempo (unidades arbitrarias)")
fig.update_yaxes(title_text = "Concentração (unidades relativas)")
fig.show()

new_xh = sympy.Matrix(xh)
new_xh

Matrix([
[               Eq(A(t), 0.497500031249413*exp(-1.01005012499922*t) + 0.502499968750585*exp(-0.990049875000781*t))],
[                Eq(B(t), -49.9993750117184*exp(-1.01005012499922*t) + 49.9993750117184*exp(-0.990049875000781*t))],
[Eq(C(t), 1.00000000000007 + 49.501874980469*exp(-1.01005012499922*t) - 50.501874980469*exp(-0.990049875000781*t))]])

In [2]:
sympy.latex(new_xh)

'\\left[\\begin{matrix}A{\\left(t \\right)} = 0.497500031249413 e^{- 1.01005012499922 t} + 0.502499968750585 e^{- 0.990049875000781 t}\\\\B{\\left(t \\right)} = - 49.9993750117184 e^{- 1.01005012499922 t} + 49.9993750117184 e^{- 0.990049875000781 t}\\\\C{\\left(t \\right)} = 1.00000000000007 + 49.501874980469 e^{- 1.01005012499922 t} - 50.501874980469 e^{- 0.990049875000781 t}\\end{matrix}\\right]'

In [3]:
#@title ## Derivada primeira { run: "auto", form-width: "20%", display-mode: "form" }
import numpy as np

def matrix(t_eval):
    return np.array([sympy.lambdify(t, sympy.diff(xh[0].rhs, t), "numpy")(t_eval),
                     sympy.lambdify(t, sympy.diff(xh[1].rhs, t), "numpy")(t_eval),
                     sympy.lambdify(t, sympy.diff(xh[2].rhs, t), "numpy")(t_eval)])

import pandas as pd

ddf = pd.DataFrame(
    map(matrix, np.linspace(0.05,5,50)),
    columns = ["dA(t)/dt", "dB(t)/dt", "dC(t)/dt"]
    )

import plotly.graph_objects as go

fig = go.Figure(layout_title_text="Gráfico {}: Variação da Concetração pelo tempo".format(options))

fig.add_trace(go.Scatter(y = ddf["dA(t)/dt"],
                    mode = "lines+markers", 
                    name = "dA(t)/dt"))

fig.add_trace(go.Scatter(y = ddf["dB(t)/dt"],
                    mode = "lines+markers", 
                    name = "dB(t)/dt"))

fig.add_trace(go.Scatter(y = ddf["dC(t)/dt"],
                    mode = "lines+markers", 
                    name = "dC(t)/dt"))

fig.update_xaxes(title_text = "tempo (unidades arbitrarias)")
fig.update_yaxes(title_text = "Variação da Concentração (unidades relativas)")
fig.show()

dxh = sympy.simplify(
    sympy.Matrix([sympy.Equality(sympy.diff(xh[0].lhs, t), sympy.diff(xh[0].rhs, t)),
                  sympy.Equality(sympy.diff(xh[1].lhs, t), sympy.diff(xh[1].rhs, t)),
                  sympy.Equality(sympy.diff(xh[2].lhs, t), sympy.diff(xh[2].rhs, t))]))
dxh

Matrix([
[Eq(Derivative(A(t), t), -0.502499968750584*exp(-1.01005012499922*t) - 0.497500031249413*exp(-0.990049875000781*t))],
[    Eq(Derivative(B(t), t), 50.5018749804689*exp(-1.01005012499922*t) - 49.501874980469*exp(-0.990049875000781*t))],
[  Eq(Derivative(C(t), t), -49.9993750117184*exp(-1.01005012499922*t) + 49.9993750117184*exp(-0.990049875000781*t))]])

In [4]:
sympy.latex(dxh)

'\\left[\\begin{matrix}\\frac{d}{d t} A{\\left(t \\right)} = - 0.502499968750584 e^{- 1.01005012499922 t} - 0.497500031249413 e^{- 0.990049875000781 t}\\\\\\frac{d}{d t} B{\\left(t \\right)} = 50.5018749804689 e^{- 1.01005012499922 t} - 49.501874980469 e^{- 0.990049875000781 t}\\\\\\frac{d}{d t} C{\\left(t \\right)} = - 49.9993750117184 e^{- 1.01005012499922 t} + 49.9993750117184 e^{- 0.990049875000781 t}\\end{matrix}\\right]'

In [5]:
#@title ## Derivada Segunda { run: "auto", form-width: "20%", display-mode: "form" }
import numpy as np

def matrix(t_eval):
    return np.array([sympy.lambdify(t, sympy.diff(xh[0].rhs, t, t), "numpy")(t_eval),
                     sympy.lambdify(t, sympy.diff(xh[1].rhs, t, t), "numpy")(t_eval),
                     sympy.lambdify(t, sympy.diff(xh[2].rhs, t, t), "numpy")(t_eval)])

import pandas as pd

dddf = pd.DataFrame(
    map(matrix, np.linspace(0.05,5,50)),
    columns = ["d²A(t)/dt²", "d²B(t)/dt²", "d²C(t)/dt²"]
    )

import plotly.graph_objects as go

fig = go.Figure(layout_title_text="Gráfico {}: Variação da Variação pelo tempo".format(options))

fig.add_trace(go.Scatter(y = dddf["d²A(t)/dt²"],
                    mode = "lines+markers", 
                    name = "d²A(t)/dt²"))

fig.add_trace(go.Scatter(y = dddf["d²B(t)/dt²"],
                    mode = "lines+markers", 
                    name = "d²B(t)/dt²"))

fig.add_trace(go.Scatter(y = dddf["d²C(t)/dt²"],
                    mode = "lines+markers", 
                    name = "d²C(t)/dt²"))

fig.update_xaxes(title_text = "tempo (unidades arbitrarias)")
fig.update_yaxes(title_text = "Variação da Variação (unidades relativas)")
fig.show()

ddxh = sympy.simplify(
    sympy.Matrix([sympy.Equality(sympy.diff(xh[0].lhs, t, t), sympy.diff(xh[0].rhs, t, t)),
                  sympy.Equality(sympy.diff(xh[1].lhs, t, t), sympy.diff(xh[1].rhs, t, t)),
                  sympy.Equality(sympy.diff(xh[2].lhs, t, t), sympy.diff(xh[2].rhs, t, t))]))
ddxh

Matrix([
[Eq(Derivative(A(t), (t, 2)), 0.507550156248631*exp(-1.01005012499922*t) + 0.492549843751366*exp(-0.990049875000781*t))],
[ Eq(Derivative(B(t), (t, 2)), -51.0094251367176*exp(-1.01005012499922*t) + 49.0093251367177*exp(-0.990049875000781*t))],
[   Eq(Derivative(C(t), (t, 2)), 50.5018749804689*exp(-1.01005012499922*t) - 49.501874980469*exp(-0.990049875000781*t))]])

In [6]:
sympy.latex(ddxh)

'\\left[\\begin{matrix}\\frac{d^{2}}{d t^{2}} A{\\left(t \\right)} = 0.507550156248631 e^{- 1.01005012499922 t} + 0.492549843751366 e^{- 0.990049875000781 t}\\\\\\frac{d^{2}}{d t^{2}} B{\\left(t \\right)} = - 51.0094251367176 e^{- 1.01005012499922 t} + 49.0093251367177 e^{- 0.990049875000781 t}\\\\\\frac{d^{2}}{d t^{2}} C{\\left(t \\right)} = 50.5018749804689 e^{- 1.01005012499922 t} - 49.501874980469 e^{- 0.990049875000781 t}\\end{matrix}\\right]'

In [7]:
#@title ## Todos os valores { run: "auto", form-width: "20%", display-mode: "form" }

import pandas as pd

pd.concat([df,ddf,dddf], axis = 1)

,A(t),B(t),C(t),dA(t)/dt,dB(t)/dt,dC(t)/dt,d²A(t)/dt²,d²B(t)/dt²,d²C(t)/dt²
0,1.000000,0.000000,7.105427e-14,-0.951225,0.903663,0.047561,0.951315,-1.854979,0.903663
1,0.902993,0.092142,4.865178e-03,-0.859818,0.729967,0.129851,0.859891,-1.589858,0.729967
2,0.815398,0.166406,1.819677e-02,-0.777196,0.581309,0.195887,0.777254,-1.358563,0.581309
3,0.736300,0.225394,3.830632e-02,-0.702514,0.454480,0.248034,0.702559,-1.157039,0.454480
4,0.664876,0.271371,6.375329e-02,-0.635009,0.346658,0.288350,0.635043,-0.981702,0.346658
5,0.600381,0.306306,9.331285e-02,-0.573991,0.255362,0.318629,0.574016,-0.829378,0.255362
6,0.542143,0.331910,1.259476e-01,-0.518836,0.178410,0.340427,0.518854,-0.697264,0.178410
7,0.489554,0.349663,1.607828e-01,-0.468982,0.113889,0.355093,0.468994,-0.582883,0.113889
8,0.442067,0.360848,1.970842e-01,-0.423919,0.060121,0.363799,0.423925,-0.484046,0.060121
9,0.399187,0.366573,2.342397e-01,-0.383186,0.015634,0.367553,0.383188,-0.398822,0.015634


In [9]:
#@title ## Parametros da Reação e Condições iniciais { run: "auto", form-width: "20%", display-mode: "form" }
#@markdown $A \rightleftharpoons {B} \rightarrow C$

#@markdown ---
#@markdown ### Condição Inicial
A_0 = 1 #@param {type:"slider", min:0.05, max:1, step:0.05}
B_0 = 0.35 #@param {type:"slider", min:0.00, max:1, step:0.05}
C_0 = 0 #@param {type:"slider", min:0.00, max:1, step:0.05}
#@markdown ---
#@markdown ### Constantes de Equilibrio
k1 = 1 #@param {type:"number"}
km1 = 0#@param {type:"number"}
k2 = 1 #@param {type:"number"}

A0 = A_0 / (A_0 + B_0 + C_0)
B0 = B_0 / (A_0 + B_0 + C_0)
C0 = C_0 / (A_0 + B_0 + C_0)

import sympy

t = sympy.symbols("t")

lamb1 = sympy.symbols("lambda1")
lamb2 = sympy.symbols("lambda2")
lamb3 = sympy.symbols("lambda3")

A = sympy.Function("A")
B = sympy.Function("B")
C = sympy.Function("C")

X = sympy.Matrix([A(t),B(t),C(t)])
dX = sympy.diff(X, t)

M = sympy.Matrix([[-k1, km1, 0], [k1, -k2-km1, 0], [0, k2, 0]])

xh = sympy.solvers.ode.dsolve(dX - M*X, ics = {A(0) : A0, B(0) : B0, C(0) : C0})

import numpy as np

def matrix(t_eval):
    return np.array([sympy.lambdify(t, xh[0].rhs, "numpy")(t_eval),
                     sympy.lambdify(t, xh[1].rhs, "numpy")(t_eval),
                     sympy.lambdify(t, xh[2].rhs, "numpy")(t_eval)])

import pandas as pd

df = pd.DataFrame(
    map(matrix, np.linspace(0.05,5,50)),
    columns = ["A(t)", "B(t)", "C(t)"]
    )

import plotly.graph_objects as go

fig = go.Figure()

fig.add_trace(go.Scatter(y = df["A(t)"],
                    mode = "lines+markers", 
                    name = "A(t)"))

fig.add_trace(go.Scatter(y = df["B(t)"],
                    mode = "lines+markers", 
                    name = "B(t)"))

fig.add_trace(go.Scatter(y = df["C(t)"],
                    mode = "lines+markers", 
                    name = "C(t)"))
fig.show()

sympy.Matrix(xh)

Matrix([
[                              Eq(A(t), 0.740740740740741*exp(-t))],
[Eq(B(t), 0.740740740740741*t*exp(-t) + 0.259259259259259*exp(-t))],
[       Eq(C(t), -0.740740740740741*t*exp(-t) + 1.0 - 1.0*exp(-t))]])